In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Armado de features**

In [67]:
auctions_w1 = pd.read_pickle('data/auctions_w1.pkl')
clicks_w1 = pd.read_pickle('data/clicks_w1.pkl')
#installs_w1 = pd.read_pickle('data/installs_w1.pkl')
events_w1 = pd.read_pickle('data/events_w1.pkl')

In [91]:
auctions_w1_label = pd.read_pickle("data/auctions_label_w1.pkl")

In [68]:
gb = auctions_w1.groupby('ref_hash')
auctions_w1['n_auctions'] = gb['date'].transform('count')
auctions_w1['last_auction'] = gb['date'].transform('max')
auctions_w1['first_auction'] = gb['date'].transform('min')
auctions_w1['diff_auctions'] = (auctions_w1['last_auction'] - auctions_w1['first_auction']).dt.total_seconds()
auctions_w1['prom_time_auction'] = 0
auctions_w1.loc[auctions_w1['n_auctions'] > 1, 'prom_time_auction'] = ((auctions_w1['last_auction'] - auctions_w1['first_auction'])/ \
                                 (auctions_w1['n_auctions'] -1)).dt.total_seconds()
auctions_w1['first_auction_sec'] = (auctions_w1['first_auction'] - pd.Timestamp('2019-04-18 00:00:00.000000')).dt.total_seconds()
auctions_w1['last_auction_sec'] = (auctions_w1['last_auction'] - pd.Timestamp('2019-04-18 00:00:00.000000')).dt.total_seconds()
auctions_w1['ref_type_id_1'] = auctions_w1['ref_type_id'].apply(lambda x: 1 if x==1 else 0)
auctions_w1['ref_type_id_7'] = auctions_w1['ref_type_id'].apply(lambda x: 1 if x==7 else 0)

In [70]:
auctions_w1.drop_duplicates(subset='ref_hash', inplace=True)
auctions_w1 = auctions_w1[['ref_hash', 'n_auctions', 'ref_type_id_1', 'ref_type_id_7', 'diff_auctions', 'prom_time_auction', 'first_auction_sec', 'last_auction_sec']]

In [72]:
gb = clicks_w1.groupby('ref_hash')
clicks_w1['n_clicks'] = gb['created'].transform('count')
clicks_w1['last_click'] = gb['created'].transform('max')
clicks_w1['first_click'] = gb['created'].transform('min')
clicks_w1['diff_clicks'] = (clicks_w1['last_click'] - clicks_w1['first_click']).dt.total_seconds()
clicks_w1['prom_time_click'] = 0
clicks_w1.loc[clicks_w1['n_clicks'] > 1, 'prom_time_auction'] = ((clicks_w1['last_click'] - clicks_w1['first_click'])/ \
                                 (clicks_w1['n_clicks'] -1)).dt.total_seconds()

In [73]:
clicks_w1['first_click_sec'] = (clicks_w1['first_click'] - pd.Timestamp('2019-04-18 00:00:00.000000')).dt.total_seconds()
clicks_w1['last_click_sec'] = (clicks_w1['last_click'] - pd.Timestamp('2019-04-18 00:00:00.000000')).dt.total_seconds()
clicks_w1['wifi_connection'] = clicks_w1['wifi_connection'].map({True: 1, False: 0})
clicks_w1['timeToClick_mean'] = clicks_w1.groupby('ref_hash')['timeToClick'].transform('mean')
clicks_w1.loc[clicks_w1.touchX == 'Infinity', 'touchX'] = 1
clicks_w1.loc[clicks_w1.touchY == 'Infinity', 'touchY'] = 1
clicks_w1["touchX"] = pd.to_numeric(clicks_w1["touchX"])
clicks_w1["touchY"] = pd.to_numeric(clicks_w1["touchY"])
clicks_w1['touchX_mean'] = clicks_w1.groupby('ref_hash')['touchX'].transform('mean')
clicks_w1['touchY_mean'] = clicks_w1.groupby('ref_hash')['touchY'].transform('mean')
clicks_w1['latitude_mean'] = clicks_w1.groupby('ref_hash')['latitude'].transform('mean')
clicks_w1['longitude_mean'] = clicks_w1.groupby('ref_hash')['longitude'].transform('mean')
clicks_w1['timeToClick_mean'] = clicks_w1['timeToClick_mean'].fillna(clicks_w1['timeToClick_mean'].mean())
clicks_w1['touchX_mean'] = clicks_w1['touchX_mean'].fillna(clicks_w1['touchX_mean'].mean())
clicks_w1['touchY_mean'] = clicks_w1['touchY_mean'].fillna(clicks_w1['touchY_mean'].mean())

In [74]:
clicks_w1.drop_duplicates(subset='ref_hash', inplace=True)
clicks_w1 = clicks_w1[['ref_hash', 'touchX_mean', 'touchY_mean', 'timeToClick_mean', 'latitude_mean', 'longitude_mean', 'n_clicks', 'wifi_connection', 'first_click_sec', 'last_click_sec', 'diff_clicks', 'prom_time_click']]

In [78]:
events_w1.drop(columns=['event_uuid', 'ip_address', 'index', 'device_countrycode'], inplace=True)
events_w1['n_events'] = events_w1.groupby('ref_hash')['date'].transform('count')
events_w1['attributed'] = events_w1['attributed'].map({True: 1, False: 0})
events_w1['wifi'] = events_w1['wifi'].map({True: 1, False: 0})
events_w1['attributed_mean'] = events_w1.groupby('ref_hash')['attributed'].transform('mean')
events_w1['wifi_mean'] = events_w1.groupby('ref_hash')['wifi'].transform('mean')
events_w1['first_event'] = events_w1.groupby('ref_hash')['date'].transform('min')
events_w1['last_event'] = events_w1.groupby('ref_hash')['date'].transform('max')
events_w1['diff_events'] = (events_w1['last_event'] - events_w1['first_event']).dt.total_seconds()
events_w1['mean_time_events'] = 0
events_w1.loc[events_w1['n_events'] > 1, 'mean_time_events'] = ((events_w1['last_event'] - events_w1['first_event'])/ \
                                 (events_w1['n_events'] -1)).dt.total_seconds()
events_w1['first_event_sec'] = (events_w1['first_event'] - pd.Timestamp('2019-04-18 00:00:00.000000')).dt.total_seconds()
events_w1['last_event_sec'] = (events_w1['last_event'] - pd.Timestamp('2019-04-18 00:00:00.000000')).dt.total_seconds()

In [80]:
events_w1.drop_duplicates(subset='ref_hash', inplace=True)
events_w1 = events_w1[['ref_hash', 'first_event_sec', 'last_event_sec', 'n_events', 'attributed_mean', 'wifi_mean', 'diff_events', 'mean_time_events']]

In [83]:
model_w1 = pd.merge(auctions_w1, clicks_w1, on='ref_hash', how='inner')

In [85]:
model_w1 = pd.merge(model_w1, events_w1, on='ref_hash', how='inner')

In [92]:
model_w1 = pd.merge(model_w1, auctions_w1_label, on='ref_hash', how='left')
model_w1['time_appearence'] = model_w1['time_appearence'].fillna(259200)

In [93]:
model_w1.to_pickle('data/model/model3_w1.pkl')